# Face Detection - Model Evaluation

**Author: Brian van den Berg**

This notebook tests and evaluates the performance of face detection models. The performance is measured by finding the amount of True Positives (TP), False Positives (FP) and False Negatives (FN). During execution, it is possible to view the evaluated images as long as the flag is raised and a UI is available to display opencv Matrices on. The reason for this research was out of personal interest and to support an assignment for a university class called 'Human Machine Interfaces'. Originally we only had to train a Haar Cascade classifier, but I wanted to benchmark our own model against the pre-made one and against another very popular relatively new classifier 'yunet'.

In [431]:
import os
import sys
import math
import cv2

## Setup

To spare you the details, in the setup, the face annotations made with the opencv_annotation tool are loaded into a list of dictionaries containing the outlines of boxes. Besides that, the functions are created to perform an evaluation and if you are interested in how I solved the issue of matching annotations with detections to get the metrics for evaluation, then I would recommend you to read the following code cells.

### Fetch Annotations

Change the path to the correct path, or an absolute one to the annotations file created using the 'opencv_annotation' tool.

In [432]:
annotations_path = os.path.join('evaluation', 'info.dat')

# Initialize an empty list to store annotations
annotations = []
num_pictures = 0

# Open the annotations file for reading
with open(annotations_path, 'r') as file:
    # Loop through the lines in the annotation file
    for i, line in enumerate(file):
        # Increment the line count
        num_pictures = i + 1

        # Split the line into values
        values = line.strip().split()

        # Check for the end of the annotations file
        if len(values) == 0:
            break

        # Get the image path and number of faces in the image
        img_path = values[0]
        face_index = 1

        # Error check the line to ensure it has the expected format
        try:
            num_faces = int(values[face_index])
        except ValueError:
            print(f'Line {i}: Warning: Skipping Line; Incorrect Format:')
            print(f'- Line: "{line.strip()}"')
            continue

        # Loop through every face in the picture
        for _ in range(num_faces):
            # Get the variables for face 'j' in the picture
            face_x = int(values[face_index + 1])
            face_y = int(values[face_index + 2])
            face_w = int(values[face_index + 3])
            face_h = int(values[face_index + 4])
            
            # Increment the face_index for the next face
            face_index += 4

            # Save the variables as a facebox
            annotation = {
                'filename': img_path,
                'x': face_x,
                'y': face_y,
                'w': face_w,
                'h': face_h
            }
            # Append the facebox to the list
            annotations.append(annotation)

# Print the result for debugging
print(f'Info: '
      f'Found {len(annotations)} face{"s" if len(annotations) > 1 else ""} '
      f'in {num_pictures} picture{"s" if num_pictures > 1 else ""}')

Info: Found 31 faces in 1 picture


### Setup Model Evaluation

As an addition to the already commented on structure, I'll also add that the 'detect_callback' arguement in 'evaluate_model' is a function callback that is called to perform face detection on an image. It is expected that the function passes a cv2.Mat type BGR image to get a list of dictionaries much like the annotations, but without 'filename' defined.

In [433]:
def calculate_center(box: dict[str, int]) -> tuple[int, int]:
    """Calculate the center of a bounding box."""
    x = round(box['x'] + box['w'] / 2)
    y = round(box['y'] + box['h'] / 2)
    return x, y

def calculate_distance(point1: tuple[int, int], point2: tuple[int, int]) -> float:
    """Calculate the Euclidean distance between two points."""
    x1, y1 = point1
    x2, y2 = point2
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

def find_matching_annotation(annotations: list[dict[str, int]], detected: dict[str, int], error_threshold: float=0.25) -> tuple[bool, list[dict[str, any]]]:
    """Find and match the closest annotation to a detected face."""
    detected_center = calculate_center(detected)

    # Find the closest label
    minimum_distance = sys.float_info.max
    minimum_index = None
    for i, annotation in enumerate(annotations):
        annotation_center = calculate_center(annotation)
        distance = calculate_distance(detected_center, annotation_center)
        if distance < minimum_distance:
            minimum_distance = distance
            minimum_index = i

    # If there were no labels, return False
    if minimum_index is None:
        return False, annotations
    
    # Calculate the distance requirement based on the size of the annotation
    annotation = annotations[minimum_index]
    allowed_distance = (annotation['h'] / 2) * error_threshold if annotation['h'] > annotation['w'] else (annotation['w'] / 2) * error_threshold

    # Check if the distance abides
    if minimum_distance < allowed_distance:
        # If distance is within the limit, remove the matched label
        del annotations[minimum_index]
        return True, annotations
    else:
        # If distance is too large, keep the label
        return False, annotations

def draw_rectangle(image: cv2.Mat, box: cv2.Mat, color: tuple[int, int, int]=(0, 255, 0), thickness: int=1) -> None:
    """Draw a rectangle on the image."""
    x, y, w, h = box['x'], box['y'], box['w'], box['h']
    cv2.rectangle(image, (x, y), (x+w, y+h), color, thickness)

def evaluate_model(detect_callback, labels: list[dict[str, any]], detection_threshold: int=0.25, show_images: bool=False) -> dict[str, dict[str, int]]:
    """Evaluate a face detection model."""
    results = {}

    # Iterate through every label
    for label in labels:
        img_path = label['filename']
        if img_path in results:
            continue

        # Setup the evaluation metrics
        TP, FP, FN = 0, 0, 0

        # Get every annotation associated with the image
        annotations = [annotation for annotation in labels if annotation.get('filename') == img_path]

        # Detect faces in the test image using the provided callback
        img = cv2.imread(img_path)
        detected = detect_callback(img)

        # Match every detected face with the annotations
        for box in detected:
            ret, annotations = find_matching_annotation(annotations, box, detection_threshold)
            if ret:
                # Detected face matches annotation (True Positive)
                draw_rectangle(img, box, (0, 255, 0), 1)
                TP += 1
            else:
                # Detected face does not match annotation (False Positive)
                draw_rectangle(img, box, (0, 255, 255), 1)
                FP += 1

        # Remaining annotations were not detected (False Negative)
        FN = len(annotations)
        for annotation in annotations:
            draw_rectangle(img, annotation, (0, 0, 255), 1)

        # Display the marked test image if requested
        if show_images:
            cv2.imshow("Image", img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        
        # Add the metrics to the result
        results[img_path] = {'TP': TP, 'FP': FP, 'FN': FN}

    return results

## Evaluation - Yunet

In [434]:
# Initialize the yunet model
yunet_model_path = os.path.join('yunet', 'models', 'face_detection_yunet_2023mar.onnx')
yunet = cv2.FaceDetectorYN.create(
    model=      yunet_model_path,
    config=     "",
    input_size= (0, 0)
)

# Define a detect callback function that delivers the result in the required format
def yunet_detect(img: cv2.Mat) -> list[dict[str, int]]:
    yunet.setInputSize((img.shape[1], img.shape[0]))
    _, faces = yunet.detect(img)
    result = []
    for face in faces:
        result.append({
            'x': int(face[0]),
            'y': int(face[1]),
            'w': int(face[2]),
            'h': int(face[3])
        })
    return result

# Perform the evaluation on yunet
evaluation_results = evaluate_model(yunet_detect, annotations, show_images=True)
print(evaluation_results)

{'evaluation\\group.jpg': {'TP': 29, 'FP': 0, 'FN': 2}}
